# ONNX Interface Demo

To be clarified:
- How to interpret onnx outputs named ...*sigmoid*...
- Where are the bounding boxes?
- Where is Michael's code for the other models FasterRCNN, ...
- Michael's thesis literature?

Demo showing how to interface YOLO with the ONNX framework.

Packages are availabe under conda environment 'yolov5'


### Load YOLO and prepare
Make sure that we are in conda environment 'yolov5' !!!


In [ ]:
from pathlib import Path
import importlib.util
#import json
import os
import ast

import yolov5
from yolov5 import YOLOv5
#from yolov5 import export  # does not work

import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw, ImageFont

import onnxruntime
import onnx
from onnx import numpy_helper


home_dir = os.getcwd()
data_dir = home_dir + '/../data'

# Import yolo's prediction app (detect.py) manually
pred_app_path = os.path.dirname(yolov5.__file__) + '/detect.py'
pred_spec = importlib.util.spec_from_file_location('detect', pred_app_path)
detect = importlib.util.module_from_spec(pred_spec)
pred_spec.loader.exec_module(detect)

# Import yolo's export app
exp_app_path = os.path.dirname(yolov5.__file__) + '/export.py'
exp_spec = importlib.util.spec_from_file_location('export', exp_app_path)
export = importlib.util.module_from_spec(exp_spec)
exp_spec.loader.exec_module(export)


# Load the trained model
#model = YOLOv5(data_dir + '/model/pen-parts/weights/best.pt')


Select model dataset:

In [ ]:
dataset_name = 'planets-gearbox'

# Image size needs to correspond to the model and should be looked up manually.
image_size = (928,928)

pt_model_path = os.path.join(data_dir, 'model', dataset_name, 'weights/best.pt')
metadata_path = os.path.join(data_dir, 'training', dataset_name, 'data.yml') 
valid_images_dir = os.path.join(data_dir, 'training', dataset_name, 'valid/images')


### Run a prediction with YOLO


In [ ]:
# Specify YOLO's prediction locations 
prediction_output_path = os.path.join(data_dir, 'prediction')

# Run with YOLO
detect.run(weights=pt_model_path, 
           source=valid_images_dir, 
           data=metadata_path,
           imgsz=image_size,
           conf_thres=0.25, 
           project=prediction_output_path, 
           name=dataset_name)


### Convert YOLO model from pt to onnx


Export using the export app

In [ ]:
# Spcify export parameters
batch_size = 1 # num of images to be processed in one run
exp_format = ['onnx']
opset_version = 10 # the ONNX version to use for export
use_dynamic = True # do not allow variable image size as input
use_simplify = True # do not use the onnx simplifier

#image_size = (900,900)

# Export with YOLO
console_out = export.run(data=metadata_path,
           weights=pt_model_path,
           imgsz=image_size,
           batch_size=batch_size,
           opset=opset_version,
           include=exp_format,
           dynamic=use_dynamic,
           simplify=use_simplify)

onnx_model_path = Path(pt_model_path).with_suffix('.onnx')

print(f"\nModel exportet to: {onnx_model_path}")



In [ ]:
%%script echo skipping
# This will be the (fixed) format for predictions with the exported model.
# Use the same format as the one for the training images.
batch_size = 1 # num of images to be processed in one run
trace_im = (torch.randn(batch_size, 3, image_size[0], image_size[1], requires_grad=False),)

file_export = Path('plantes-gearbox.onnx') # filename as pathlib object
opset_version = 10 # the ONNX version to use for export
use_dynamic = False # do not allow variable image size as input
use_simplify = False # do not use the onnx simplifier

# Export the model to ONNX
export.export_onnx(model.model, trace_im, file_export, opset_version, use_dynamic, use_simplify) 



### Run a prediction with onnx runtime

Define some helpers:

In [ ]:
def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
        
    #add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, image_size[0], image_size[1]).astype('float32')
    return norm_img_data

def softmax(x):
    x = x.reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def postprocess(result):
    return softmax(np.array(result)).tolist()

Load model and runtime

In [ ]:
session = onnxruntime.InferenceSession(onnx_model_path, None)

# get the name of the first input of the model
input_name = session.get_inputs()[0].name
labels_string = session.get_modelmeta().custom_metadata_map['names']
labels = ast.literal_eval(labels_string)
model_output = session.get_outputs()

#print('Input name:', input_name)

print("Input names: ", end='')
for inp in session.get_inputs(): print(inp.name, " ", end='')

print(f"\nLabels: {list(labels.values())}")

print("Ouput names: ", end='')
for out in model_output:
    print(out.name, " ", end='')


Load in one of the example images:

In [ ]:
#labels = load_labels('imagenet-simple-labels.json')

#example_image_name = '/Video1_8_jpg.rf.815eb1fbb012e439b92e7c0c7cb93b0f.jpg'
example_image_name = '/1_928.jpg'

image = Image.open(valid_images_dir + example_image_name)

plt.axis('off')
plt.imshow(image)
plt.show()
#plt.imshow(image)
print(f"Image size: {image.size}")

image_data = np.array(image).transpose(2, 0, 1)
input_data = preprocess(image_data)

print(f"Image shape: {image_data.shape}")


Run the prediction

In [ ]:
raw_result = session.run([], {input_name: input_data})
res = postprocess(raw_result[0][0][0].tolist())

# Analyse output data structure
print("Output data format:")
for i in [0,1,2,3]:
    print(f"{model_output[i].name}: {raw_result[i].shape}")

print(raw_result[0][0].shape)

raw_result[0][0].transpose()

Interprete inference

In [ ]:
idx = np.argmax(res)

print('Final top prediction is: ' + labels[idx])
#print('\n')

sort_idx = np.flip(np.squeeze(np.argsort(res)))
print('Top 5 labels are:')
print(list(map(labels.get, sort_idx[:5])))